<a href="https://colab.research.google.com/github/AbhilashDatta/Role-Identification-in-Law-Documents/blob/main/Coreference_Resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install folium==0.2.1

In [ ]:
!pip install urllib3==1.25.4

In [ ]:
!pip install neuralcoref

In [ ]:
!pip install spacy==2.1.0

In [ ]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import spacy
from spacy.attrs import ORTH
import re
import glob
from bs4 import BeautifulSoup

In [ ]:

def custom_sentencizer(doc):
    ''' Look for sentence start tokens by scanning for periods only. '''
    for i, token in enumerate(doc[:-2]):  # The last token cannot start a sentence
        if token.text == ".":
            #doc[i+1].is_sent_start = True
            pass
        else:
            doc[i+1].is_sent_start = False  # Tell the default sentencizer to ignore this token
    return doc

nlp = spacy.load("en_core_web_lg")
nlp.add_pipe(custom_sentencizer, before = "parser")

with open("/content/drive/MyDrive/abbreviations.txt") as fr:
    abbv = fr.read().strip().split('\n')

special_cases = {a: a for a in abbv} 
#"S.": "section", "s.": "section", "ss.": "section", "u/s.": "section", "u/ss.": "section", "art.": "article"}

for case, orth in special_cases.items():
    nlp.tokenizer.add_special_case(case, [{ORTH: orth}])

In [ ]:
def getSentList(Text):
#     print(Text)
    _RE_COMBINE_WHITESPACE = re.compile(r"\s+")

    ''' Gives the list of clean sentences from the html text '''
    cleanText = Text.text.strip().replace('\n', ' ')
    cleanText = _RE_COMBINE_WHITESPACE.sub(" ", cleanText).strip()
    #print(cleanText)
    parsed = nlp(cleanText)
    sentList=[]
    for sent in parsed.sents:
        sentList.append(sent.text)
    return sentList  

In [ ]:
def getPointText(dict,pointText,pointNumber):
    ''' Stores the text under a point into a dictionary with key as point number and value as nested dictionary of splitted sentences '''
    soup = BeautifulSoup(pointText, 'html.parser')
    dict[pointNumber]={}
    for pText in soup.find_all('p'):
        dict[pointNumber][pText.get('id')] = getSentList(pText)     
    for spanText in soup.findAll('span'):
        dict[pointNumber][spanText.get('id')] = getSentList(spanText)
    for blockquoteText in soup.findAll('blockquote'):
        dict[pointNumber][blockquoteText.get('id')] = getSentList(blockquoteText) 

In [ ]:
''' Files should be stored in the format of docid.html '''
  
def process(fname):
    print("Case docid:", end = ' ')
    # fname = input()
    # fname = "185297023"
    print(fname)
    docids = [str(fname)+".html"]

    resultDict={}

    for docid in docids:
        #print("\n\nDOCID = "+str(docid)+"\n\n")
        with open("/content/drive/MyDrive/Annot_Facts/" + docid,'r',encoding="utf-8") as f:
            count = 1
            pointText = ""
            dict = {}
            for line in f:
                #line=' <p id="p_1">       1. " Ramu is a good boy'
                #line='</p><p id="p_41">                “15. In our opinion, the measure of'
                text = re.search('(.*?)<p id="p_[0-9]+">(((?!\s*“\s*).)*?)'+str(count)+'[.]\s(.*?)',line)
                if text is not None:
                    #print(pointText)
                    getPointText(dict,pointText,count-1)
                    pointText = line
                    count = count+1
                else:
                    text = re.search('(.*?)<p id="p_[0-9]+">(((?!\s*“\s*).)*?)'+str(count+1)+'[.]\s(.*?)',line)
                    if text is not None:
                        #print(pointText)
                        getPointText(dict,pointText,count-1)
                        pointText = line
                        count = count+2
                    else:
                        text = re.search('(.*?)<p id="p_[0-9]+">(((?!\s*“\s*).)*?)'+str(count+2)+'[.]\s(.*?)',line)
                        if text is not None:
                            #print(pointText)
                            getPointText(dict,pointText,count-1)
                            pointText = line
                            count = count+3
                        else:
                            text = re.search('(.*?)<p id="p_[0-9]+">(((?!\s*“\s*).)*?)'+str(count+3)+'[.]\s(.*?)',line)
                            if text is not None:
                                #print(pointText)
                                getPointText(dict,pointText,count-1)
                                pointText = line
                                count = count+4
                            else:
                                pointText = pointText+line
        getPointText(dict,pointText,count-1)
        resultDict[docid[:-5]] = dict  
    
    return resultDict

         

In [ ]:
def transform(s):
  count = 0
  for c in s:
    if c.isupper():
      count += 1

  if count>= 0.75*len(s):
    s = s.lower()
  return str(s)

In [ ]:
import pprint

fname = 'sc10/10807134'
resDict = process(fname)
pprint.pprint(resDict[fname], indent = 1)

Case docid: sc10/10807134
{0: {'p_1': ['R.F. Nariman, J.']},
 1: {'p_2': ['1.', 'Leave granted.']},
 2: {'blockquote_1': ['“I am a resident of above address and doing the '
                      'business of furniture at Nehru Complex, Amritsar.',
                      'I do my religious services in the Nirankari Bhawan at '
                      'Rajasansi every Signature Not Verified Digitally signed '
                      'by Sunday.',
                      'Today, i.e, on 18.11.2018, Satsang was going on '
                      'Charanjeet kaur Date: 2020.10.12 17:45:12 IST at '
                      'Satsang Bhawan, where about 200 Satsangis were present.',
                      'At about 11.30 a.m., I along with my companion Reason:'],
     'blockquote_2': ['Gagandeep Singh son of Balwinder Singh, resident of 1 '
                      'Gumtala, was doing the duty of a Security Guard on the '
                      'main gate, when two young boys came there on a Pulsar '
         

In [ ]:
count = 1
paragraphs = []
next_to_full = []
full = ""
linelist = []

for i in resDict.keys():
  
  td1 = resDict[i]
  for j in td1.keys():
    td2 = td1[j]
    #print('Paragraph '+str(count)+":\n")
    count += 1
    x = ""
    for k in td2.keys():
      lines = ""
      for line in td2[k]:
        if(len(line)>5):
          line = transform(line)
          full += line + ' '
          x += line + ' '
          lines += line + ' ' 
          linelist.append(line)
        #print(line)
      paragraphs.append(lines)
    next_to_full.append(x)
    full += lines
    

In [ ]:
#NER Cell

import pandas as pd
from spacy import displacy

nlp = spacy.load("en_core_web_lg")

#doc = nlp(full)
#print(paragraphs[3])

entities = []
labels = []
position_start = []
position_end = []

for p in paragraphs:
  
  doc = nlp(p)

  for ent in doc.ents:
      entities.append(ent)
      labels.append(ent.label_)
      position_start.append(ent.start_char)
      position_end.append(ent.end_char)
      
  if len(doc.ents)!=0:
    displacy.render(doc, style='ent', jupyter=True)
    print()

df = pd.DataFrame({'Entities':entities,'Labels':labels,'Position_Start':position_start, 'Position_End':position_end})

#df

In [ ]:
import neuralcoref
neuralcoref.add_to_pipe(nlp)

100%|██████████| 40155833/40155833 [00:04<00:00, 8782449.21B/s] 


In [ ]:
#line wise corefs

line_corefs = []
for line in linelist:
  doc = nlp(line)
  line_corefs.append(doc._.coref_clusters)

line_corefs

#line wise coref is losing information

[[],
 [],
 [],
 [He: [He, his, his]],
 [The young boy who had gone inside the Satsang Hall: [The young boy who had gone inside the Satsang Hall, his]],
 [],
 [Both the young men: [Both the young men, their]],
 [],
 [Amritsar: [Amritsar, Amritsar, Amritsar]],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Jean: [Jean, his, he, his]],
 [],
 [],
 [],
 [Order: [Order, his]],
 [the proposition of law: [the proposition of law, it]],
 [],
 [],
 [],
 [i.e. without its jurisdiction: [i.e. without its jurisdiction, its]],
 [],
 [],
 [the Trial Court: [the Trial Court, this court]],
 [],
 [the order dated 25.02.2019: [the order dated 25.02.2019, the order]],
 [],
 [],
 [],
 [the petitioner: [the petitioner, the petitioner],
  the Hon'ble Supreme Court: [the Hon'ble Supreme Court, Exclusive Court]],
 [],
 [the UAPA: [the UAPA, the UAPA]],
 [],
 [An order that is passed without jurisdiction by the learned Sub-Divisional Judicial Magistrate dated 13.02.2019: [An order that is passed without jurisdic

In [ ]:
#Paragraph wise corefs

para_corefs = []
for p in paragraphs:
  doc = nlp(p)
  if doc._.has_coref:
    #print('Clusters:')
    #print(doc._.coref_clusters)
    para_corefs.append(doc._.coref_clusters)
    #displacy.render(doc, style='ent', jupyter=True)

para_corefs
  

[[He: [He, his, his],
  The young boy who had gone inside the Satsang Hall: [The young boy who had gone inside the Satsang Hall, his, the above-said young boy],
  the young men: [the young men, their],
  about 22 persons from the Sangat: [about 22 persons from the Sangat, The other persons],
  the Sangat: [the Sangat, the Sangat],
  Amritsar: [Amritsar, Amritsar, Amritsar],
  my statement: [my statement, It]],
 [two young boys: [two young boys, them], Jean: [Jean, his, he, his]],
 [the learned Sub-Divisional Magistrate: [the learned Sub-Divisional Magistrate, the learned Sub-Divisional Judicial Magistrate],
  an application for default bail: [an application for default bail, This application],
  an order dated 13.02.2019: [an order dated 13.02.2019, this Order]],
 [the proposition of law: [the proposition of law, it]],
 [(2) Unlawful Activities (Prevention) Act 1967: [(2) Unlawful Activities (Prevention) Act 1967, It, its],
  the said reason order passed by Ilaqa magistrate: [the said 

In [ ]:
# next_to_full hierarchy corefs

ntf_corefs = []

for p in next_to_full:
  doc = nlp(p)
  if doc._.has_coref:
    #print('Clusters:')
    #print(doc._.coref_clusters)
    ntf_corefs.append(doc._.coref_clusters)
    #displacy.render(doc, style='ent', jupyter=True)

ntf_corefs

[[He: [He, his, his],
  The young boy who had gone inside the Satsang Hall: [The young boy who had gone inside the Satsang Hall, his, the above-said young boy],
  the young men: [the young men, their],
  about 22 persons from the Sangat: [about 22 persons from the Sangat, The other persons],
  the Sangat: [the Sangat, the Sangat],
  Amritsar: [Amritsar, Amritsar, Amritsar, Amritsar],
  Rajasansi: [Rajasansi, Rajasansi],
  my statement: [my statement, It],
  two young boys: [two young boys, them],
  Jean: [Jean, his, he, his]],
 [the learned Sub-Divisional Magistrate: [the learned Sub-Divisional Magistrate, the learned Sub-Divisional Judicial Magistrate],
  an application for default bail: [an application for default bail, This application],
  an order dated 13.02.2019: [an order dated 13.02.2019, this Order],
  the proposition of law: [the proposition of law, it]],
 [(2) Unlawful Activities (Prevention) Act 1967: [(2) Unlawful Activities (Prevention) Act 1967, It, its],
  the said reas

In [ ]:
#full doc corefs
doc = nlp(full)
full_corefs = doc._.coref_clusters
full_corefs

[He: [He, his, his],
 The young boy who had gone inside the Satsang Hall: [The young boy who had gone inside the Satsang Hall, his, the above-said young boy],
 the young men: [the young men, their],
 about 22 persons from the Sangat: [about 22 persons from the Sangat, The other persons],
 the Sangat: [the Sangat, the Sangat],
 Amritsar: [Amritsar, Amritsar, Amritsar, Amritsar],
 Rajasansi: [Rajasansi, Rajasansi],
 my statement: [my statement, It],
 the duty of a Security Guard on the main gate: [the duty of a Security Guard on the main gate, the duty of a Security Guard on the main gate],
 the main gate: [the main gate, the main gate],
 two young boys: [two young boys, them, two young boys, them],
 a Pulsar Motor Cycle: [a Pulsar Motor Cycle, a Pulsar Motor Cycle],
 Jean: [Jean, his, he, his, Jean, his, he, his],
 Jean and Jacket: [Jean and Jacket, Jean and Jacket],
 his face: [his face, his face],
 the learned Sub-Divisional Magistrate: [the learned Sub-Divisional Magistrate, the lear

In [ ]:
len(full_corefs)

305

In [ ]:
count = 0
for l in para_corefs:
  count +=len(l)

count

237

In [ ]:
import pandas as pd

df = pd.DataFrame({'users':['user1','user2'], 'message': ['msg1','msg2']})

df

,users,message
0,user1,msg1
1,user2,msg2


In [ ]:
for i in range(len(df)):
  user = df.iloc[i]['users']
  message = df.iloc[i]['message']

  print(user,message)

user1 msg1
user2 msg2


In [ ]:
if 'message' in df.columns:
  print('yes')

yes


In [ ]:
df['users'].tolist()

['user1', 'user2']

In [ ]:
!git clone https://github.com/RaphaelMeudec/deblur-gan.git

Cloning into 'deblur-gan'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 168 (delta 9), reused 0 (delta 0), pack-reused 149
Receiving objects: 100% (168/168), 81.09 MiB | 28.78 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
%cd deblur-gan

/content/deblur-gan


In [ ]:
!pip install -r requirements/requirements.txt
!pip install -e .

     |████████████████████████████████| 79 kB 4.9 MB/s 
     |████████████████████████████████| 71 kB 11.2 MB/s 
     |████████████████████████████████| 264 kB 35.2 MB/s 
     |████████████████████████████████| 889 kB 39.2 MB/s 
     |████████████████████████████████| 3.3 MB 24.8 MB/s 
     |████████████████████████████████| 319 kB 60.9 MB/s 
     |████████████████████████████████| 78 kB 9.4 MB/s 
     |████████████████████████████████| 36.2 MB 74 kB/s 
     |████████████████████████████████| 1.6 MB 41.8 MB/s 
     |████████████████████████████████| 4.9 MB 76.2 MB/s 
     |████████████████████████████████| 2.1 MB 59.6 MB/s 
ERROR: Could not find a version that satisfies the requirement pkg-resources==0.0.0 (from versions: none)
ERROR: No matching distribution found for pkg-resources==0.0.0
Obtaining file:///content/deblur-gan
  Running setup.py develop for deblurgan


In [ ]:
!python scripts/deblur_image.py --weight_path=/content/deblur-gan/generator.h5 --input_dir=/content/ --output_dir=/content/


Traceback (most recent call last):
  File "scripts/deblur_image.py", line 37, in <module>
    deblur_command()
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "scripts/deblur_image.py", line 33, in deblur_command
    return deblur(weight_path, input_dir, output_dir)
  File "scripts/deblur_image.py", line 12, in deblur
    g = generator_model()
  File "/content/deblur-gan/deblurgan/model.py", line 30, in generator_model
    x = ReflectionPadding2D((3, 3))(inputs)
  File "/content/deblur-gan/deblurgan/layer_utils.py", line 134, in __init__
    height_p

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-gpu

     |████████████████████████████████| 458.3 MB 7.8 kB/s 
     |████████████████████████████████| 462 kB 63.4 MB/s 
     |████████████████████████████████| 5.6 MB 23.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.1.0
    Uninstalling tensorflow-2.1.0:
      Successfully uninstalled tensorflow-2.1.0
     |████████████████████████████████| 458.3 MB 11 kB/s 
